In [1]:
!pip install -qqq accelerate peft bitsandbytes
!pip install -qqq git+https://github.com/huggingface/transformers
!pip install -qqq trl #py7zr auto-gptq optimum
!pip install -qqq wandb -U

In [2]:
# import os

# # Disable W&B
# os.environ["WANDB_DISABLED"] = "true"

In [3]:
#udpate datasets to avoid SchemaInferece related error
!pip install -qqq -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,

In [4]:
# import gdown 
# url = 'https://drive.google.com/uc?id=15VkJdq5eyWIkfb_aoD3oS8i4tScbHYky ' 
# output = 'medmcqa.zip'
# gdown.download(url, output)

# !gdown --id <File ID>
# !gdown --id 15VkJdq5eyWIkfb_aoD3oS8i4tScbHYky

In [5]:
# from google.colab import drive
# drive.mount("/content/drive")

In [6]:
data_path='/kaggle/input/medmcqa-dataset'

In [7]:
import json
import pandas as pd

path = "/kaggle/input/medmcqa-dataset/train.json"

data_json = []
with open(path, 'r') as file:
    for line in file:
        data_json.append(json.loads(line))

data_extracted = [
    {'question': entry['question'], 'cop': entry['cop'], 'subject_name': entry['subject_name'], 'topic_name': entry['topic_name'], 'exp': entry['exp'], 'opa': entry['opa'], 'opb': entry['opb'], 'opc': entry['opc'], 'opd': entry['opd'], 'exp': entry['exp']}
    for entry in data_json
]

data_df = pd.DataFrame(data_extracted)

data_df['topic_name'] = data_df['topic_name'].fillna('Unknown')
data_df['exp'] = data_df['exp'].fillna('')

In [8]:
#!less /content/drive/MyDrive/medmcqa-data/train.json

In [9]:
data_df.head()

,question,cop,subject_name,topic_name,exp,opa,opb,opc,opd
0,Chronic urethral obstruction due to benign pri...,3,Anatomy,Urinary tract,Chronic urethral obstruction because of urinar...,Hyperplasia,Hyperophy,Atrophy,Dyplasia
1,Which vitamin is supplied from only animal sou...,3,Biochemistry,Vitamins and Minerals,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...,Vitamin C,Vitamin B7,Vitamin B12,Vitamin D
2,All of the following are surgical options for ...,4,Surgery,Surgical Treatment Obesity,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba...",Adjustable gastric banding,Biliopancreatic diversion,Duodenal Switch,Roux en Y Duodenal By pass
3,Following endaerectomy on the right common car...,1,Ophthalmology,Unknown,The central aery of the retina is a branch of ...,Central aery of the retina,Infraorbital aery,Lacrimal aery,Nasociliary aretry
4,Growth hormone has its effect on growth through?,2,Physiology,Unknown,"Ans. is 'b' i.e., IGI-1GH has two major functi...",Directly,IG1-1,Thyroxine,Intranuclear receptors


In [10]:
def generate_prompt(x):
    cop = 'Nothing'
    if x['cop'] == 1:
        cop = x['opa']
    elif x['cop'] == 2:
        cop = x['opb']
    elif x['cop'] == 3:
        cop = x['opc']
    elif x['cop'] == 4:
        cop = x['opd']
    question = '{}\nOptions:\n1. {}\n2. {}\n3. {}\n4. {}\n'.format(x['question'], x['opa'], x['opb'], x['opc'], x['opd'])
    answer = cop
    prompt = f"""<s> [INST] Solve this post graduate medical entrance exam MCQ question and provide the correct option. [/INST]
    ### Question: {question}
    ### Answer: {answer} </s>"""
    return prompt

In [11]:
import torch
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:

data = Dataset.from_pandas(data_df)

#model_id = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
model_id='mistralai/Mistral-7B-Instruct-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left' # takes less memory than 'right' padding


# quantization_config_loading = GPTQConfig(
#                                 bits=4,
#                                 disable_exllama=True,
#                                 tokenizer=tokenizer
#                             )
quantization_config_loading = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
base_model = AutoModelForCausalLM.from_pretrained(
                                model_id,
                                quantization_config=quantization_config_loading,
                                device_map="auto"
                            )

base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.use_cache=False
base_model.config.pretraining_tp=1
base_model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(base_model)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:

#del model
def clear_cache():
    import gc
    gc.collect()

    torch.cuda.empty_cache()
    #nvidia-smi

clear_cache()
!nvidia-smi

Sat Jan 20 08:02:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0              27W /  70W |   2027MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
peft_config = LoraConfig(
                    r=16,
                    lora_alpha=32,
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM",
                    target_modules=[
                        "q_proj",
                        "k_proj",
                        "v_proj",
                        "o_proj",
                        "gate_proj",
                        "up_proj",
                        "down_proj",
                        "lm_head",
                    ]
                )
model = prepare_model_for_kbit_training(base_model)
peft_model = get_peft_model(model, peft_config)

In [15]:
#enable parallel processing for dual GPU
if torch.cuda.device_count() > 1: # If more than 1 GPU
    peft_model.is_parallelizable = True
    peft_model.model_parallel = True

In [16]:
#use accelerate
#peft_model = accelerator.prepare_model(peft_model)

In [17]:
# # #for retraining
# del trainer
# del merged_model

In [18]:
from transformers import Trainer
training_arguments = TrainingArguments(
                            output_dir="mistral-7B-qlora-finetuned-medmcqa",
                            per_device_train_batch_size=4,
                            gradient_accumulation_steps=1,
                            optim="paged_adamw_32bit", #paged_adamw_32bit
                            learning_rate=1e-5,
                            lr_scheduler_type="constant",
                            # max_grad_norm=0.3,
                            # warmup_ratio=.03,
                            save_strategy="epoch",
                            logging_steps=10,
                            num_train_epochs=3,
                            max_steps=500,
                            fp16=True,
                            report_to="wandb"
                            #push_to_hub=True
                        )

trainer = SFTTrainer(
            model=peft_model,
            train_dataset=data,
            peft_config=peft_config,
            dataset_text_field="question",
            args=training_arguments,
            tokenizer=tokenizer,
            packing=False,
            max_seq_length=512
    )

# trainer = Trainer(
#     model=model,
#     train_dataset=data,
#     #peft_config=peft_config,
#     args=training_arguments,
#     tokenizer=tokenizer
# )

Map:   0%|          | 0/182822 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [19]:
clear_cache()
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Jan 20 08:02:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0              27W /  70W |   2341MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
import wandb, os
wandb.login()

wandb_project = "mistral-7B-qlora-medmcqa-kgl"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [21]:
%time trainer_stats=trainer.train()

wandb: Currently logged in as: ageek. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,4.696100
20,4.822800
30,4.469700
40,4.305800
50,4.224000
60,4.107200
70,4.194100
80,3.781600
90,3.435900
100,3.043400


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


CPU times: user 11min 6s, sys: 4min 28s, total: 15min 34s
Wall time: 16min 9s


In [22]:
path = "/kaggle/input/medmcqa-dataset/dev.json"
val_data_json = []
with open(path, 'r') as file:
    for line in file:
        val_data_json.append(json.loads(line))

val_data_extracted = [
    {'question': entry['question'], 'cop': entry['cop'], 'opa': entry['opa'], 'opb': entry['opb'], 'opc': entry['opc'], 'opd': entry['opd']}
    for entry in val_data_json
]

val_data_df = pd.DataFrame(val_data_extracted)

In [23]:
val_data_df[50:53]

,question,cop,opa,opb,opc,opd
50,What is the usual weight of rabbit used in oph...,2,0.5-1 kg,1.5-2.5 kg,5-7 kg,10-12 kg
51,"In plasmodium vivax malaria, relapse is caused...",3,Sporozoite,Schizont,Hypnozoite,Gamteocyte
52,Retraction of mandible is achieved by:,2,Lateral pterygoid,Temporalis,Medial pterygoid,Masseter


In [24]:
def generate_test_prompt(x):
    question = '{}\nOptions:\n1. {}\n2. {}\n3. {}\n4. {}\n'.format(x['question'], x['opa'], x['opb'], x['opc'], x['opd'])
    prompt = f"""
    Question:
    {question}
    [INST] Solve this post graduate medical entrance exam MCQ and answer correctly. [/INST]
    Answer: """ 
    return prompt
# val_data_df['text'] = val_data_df.apply(lambda x: generate_test_prompt(x), axis=1)

In [25]:
val_data_df['text'] = val_data_df.apply(lambda x: generate_test_prompt(x), axis=1)

In [26]:
val_data_df[50:53]

,question,cop,opa,opb,opc,opd,text
50,What is the usual weight of rabbit used in oph...,2,0.5-1 kg,1.5-2.5 kg,5-7 kg,10-12 kg,\n Question:\n What is the usual weight ...
51,"In plasmodium vivax malaria, relapse is caused...",3,Sporozoite,Schizont,Hypnozoite,Gamteocyte,\n Question:\n In plasmodium vivax malar...
52,Retraction of mandible is achieved by:,2,Lateral pterygoid,Temporalis,Medial pterygoid,Masseter,\n Question:\n Retraction of mandible is...


In [27]:
#enable full column print in pd
pd.set_option('display.max_colwidth', None)

In [28]:
val_data_df[50:53]['text']

50    \n    Question:\n    What is the usual weight of rabbit used in ophthalmological experiments?\nOptions:\n1. 0.5-1 kg\n2. 1.5-2.5 kg\n3. 5-7 kg\n4. 10-12 kg\n\n    [INST] Solve this post graduate medical entrance exam MCQ and answer correctly. [/INST]\n    Answer: 
51      \n    Question:\n    In plasmodium vivax malaria, relapse is caused by:September 2012\nOptions:\n1. Sporozoite\n2. Schizont\n3. Hypnozoite\n4. Gamteocyte\n\n    [INST] Solve this post graduate medical entrance exam MCQ and answer correctly. [/INST]\n    Answer: 
52                   \n    Question:\n    Retraction of mandible is achieved by:\nOptions:\n1. Lateral pterygoid\n2. Temporalis\n3. Medial pterygoid\n4. Masseter\n\n    [INST] Solve this post graduate medical entrance exam MCQ and answer correctly. [/INST]\n    Answer: 
Name: text, dtype: object

In [29]:
import time
#save finetuned model
peft_model_ft=f"./medmcqa-checkpoint-{str(int(time.time()))}"

trainer.model.save_pretrained(peft_model_ft)
tokenizer.save_pretrained(peft_model_ft)
#trainer.save_model("Mistral_7B_instruct_MedMCQnA")

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('./medmcqa-checkpoint-1705740506/tokenizer_config.json',
 './medmcqa-checkpoint-1705740506/special_tokens_map.json',
 './medmcqa-checkpoint-1705740506/tokenizer.model',
 './medmcqa-checkpoint-1705740506/added_tokens.json',
 './medmcqa-checkpoint-1705740506/tokenizer.json')

In [30]:
# import locale
# def getpreferredencoding(do_setlocale=True):
#     return "UTF-8"

# locale.getpreferredencoding = getpreferredencoding

# !ls -l
# !ls -la ./Mistral_7B_instruct_MedMCQnA

In [31]:
merged_model = peft_model.merge_and_unload()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [32]:
merged_model.get_memory_footprint()/1024/1024/1024

4.727561950683594

In [33]:
#load save model
# from peft import PeftModel, PeftConfig

# peft_base_model = AutoPeftModelForCausalLM.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1', torch_dtype=torch.bfloat16)
# tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1')

# peft_model_ft_saved = PeftModel.from_pretrained(
#     peft_base_model,
#     peft_model_ft,
#     torch_dtype=torch.bfloat16,
#     is_trainable=False
# )

In [34]:
# #Inference
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
import tqdm
import json
import pandas as pd

In [35]:
#clear up RAM
clear_cache()

!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Jan 20 08:49:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              32W /  70W |   2715MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [36]:
# def generate_response(prompt, model):
#   encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
#   model_inputs = encoded_input.to('cuda')

#   generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

#   decoded_output = tokenizer.batch_decode(generated_ids)

#   return decoded_output[0]

In [37]:
# #create a test prompt
# test_prompt=val_data_df[50:51]['text'].astype(str)
# test_prompt.astype(str)

In [38]:
# #Using fine tuned PEFT model
# generate_response(test_prompt, merged_model)

In [39]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.pad_token_id
)

In [40]:
idx=120
example = val_data_df['text'][idx]
inputs = tokenizer(example, return_tensors="pt").to("cuda")

outputs = merged_model.generate(**inputs, generation_config=generation_config)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer)
print(f"Actual answer: {val_data_df['cop'][idx]}")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



    Question:
    Malaria relapse common with which type plasmodium species?
Options:
1. Plasmodium malariae and vivax
2. Plasmodium falciparum and vivax
3. Plasmodium vivax and ovale
4. Plasmodium ovale and malariae

    [INST] Solve this post graduate medical entrance exam MCQ and answer correctly. [/INST]
    Answer: 1. Plasmodium malariae and vivax

    Explanation: Malaria relapse is common
Actual answer: 3


In [41]:
#Evaluation Metrics Process
def solve_question(question_prompt, model):
    inputs = tokenizer(question_prompt,
                       return_tensors="pt",
                       padding=True,
                       truncation=True).to("cuda")

    outputs = model.generate(**inputs,
                             generation_config=generation_config)

    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer


In [42]:
#Create a small validation set for quick evaluation
val_data_df_mini = val_data_df[:100]
len(val_data_df_mini)

100

In [43]:
# import tqdm
# import time

# for i in tqdm.tqdm(range(100)):
#     time.sleep(0.5)  #actual task , say trainer.train() etc.

In [44]:
clear_cache()
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Jan 20 08:49:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0              33W /  70W |   2715MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [45]:
all_answers = []
jump=1
import re
val_data_prompts = list(val_data_df_mini['text'])
for i in tqdm.tqdm(range(0, len(val_data_prompts), jump)):
    question_prompts = val_data_prompts[i:i+jump]
    ans = solve_question(question_prompts, merged_model)
    #print(f'{i}th answer: {ans}')
    ans_option = []
    for text in ans:
        #ans_option.append(re.search(r'Answer: \s*(.*)', text).group(1))
        ans_option.append(re.search(r'Answer: ?(\d*\.)? (.*)', text).group(2))

    all_answers.extend(ans_option)

100%|██████████| 100/100 [28:53<00:00, 17.34s/it]


In [46]:
all_answers[:10]

['Local anesthesia is effective only when the nerve is not covered by myelin sheath',
 'Glucose concentration in the capillaries is the same as that in glomerular filtrate.',
 'No test is required now as her age is below 35 years.',
 'Antegrade',
 'Ketogenesis',
 '0.02',
 'Pregnant woman with sore throat can be staed immediately on oseltamivir without diagnostic testing under category',
 'Septal branch of superior labial aery',
 'Ventricular bigeminy',
 'Tetralogy of fallot']

In [47]:
#val_data_df_mini[:1]

In [48]:
correct_answers = []
for i in range(len(val_data_df_mini)):
    if val_data_df_mini['cop'][i] == 1:
        correct_answers.append(val_data_df_mini['opa'][i])
    elif val_data_df_mini['cop'][i] == 2:
        correct_answers.append(val_data_df_mini['opb'][i])
    elif val_data_df_mini['cop'][i] == 3:
        correct_answers.append(val_data_df_mini['opc'][i])
    elif val_data_df_mini['cop'][i] == 4:
        correct_answers.append(val_data_df_mini['opd'][i])


correct_count = 0
for i in range(len(val_data_df_mini)):
    correct_count += correct_answers[i] == all_answers[i]
print(correct_count/len(val_data_df_mini))
# .28 with full char by char matching

0.28


In [61]:
#no char-level matchin, as answers may be worded slightly
#differently by LLMs
threshold_score = 0.6  #(say 60% is good enough)
matching_scores = [sum(g == t for g, t in zip(gen, truth)) / max(len(gen), len(truth)) for gen, truth in zip(all_answers, correct_answers)]
correct_count = sum(score >= threshold_score for score in matching_scores)
correct_count/len(val_data_df_mini)
#0.02 - only 2% answers match

0.45

In [50]:
#.45  -thereshold .6
# .42 - threshold .7
# .47 - threshold .5


In [51]:
#match analsysi for char by char matching
correct_count = 0
for i in range(len(val_data_df_mini)):
    if correct_answers[i]==all_answers[i]:
        print(f'{i}th answer is correct, correct ans:{correct_answers[i]}, and predicted_ans:{all_answers[i]}')
    correct_count += correct_answers[i] == all_answers[i]
correct_count/len(val_data_df_mini)

10th answer is correct, correct ans:Dentinoenamel junction, and predicted_ans:Dentinoenamel junction
11th answer is correct, correct ans:Before performing the ABG, syringe should be loaded with 0.3 cc of heparin, and predicted_ans:Before performing the ABG, syringe should be loaded with 0.3 cc of heparin
22th answer is correct, correct ans:Upper Canine, and predicted_ans:Upper Canine
28th answer is correct, correct ans:Collagenase, and predicted_ans:Collagenase
29th answer is correct, correct ans:Student's T-test, and predicted_ans:Student's T-test
33th answer is correct, correct ans:ABCDE, and predicted_ans:ABCDE
35th answer is correct, correct ans:Convalescent carrier, and predicted_ans:Convalescent carrier
37th answer is correct, correct ans:Gram (–)ve sepsis, and predicted_ans:Gram (–)ve sepsis
42th answer is correct, correct ans:All, and predicted_ans:All
45th answer is correct, correct ans:B, and predicted_ans:B
51th answer is correct, correct ans:Hypnozoite, and predicted_ans:Hy

0.28

In [52]:
#mismatch analysis 
correct_count = 0
for i in range(len(val_data_df_mini)):
    if correct_answers[i]!=all_answers[i]:
        print(f'{i}th answer is in-correct, correct ans:{correct_answers[i]}, and predicted_ans:{all_answers[i]}')
    correct_count += correct_answers[i] == all_answers[i]
1-correct_count/len(val_data_df_mini)

0th answer is in-correct, correct ans:Impulse through myelinated fibers is slower than non-myelinated fibers, and predicted_ans:Local anesthesia is effective only when the nerve is not covered by myelin sheath
1th answer is in-correct, correct ans:The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it, and predicted_ans:Glucose concentration in the capillaries is the same as that in glomerular filtrate.
2th answer is in-correct, correct ans:Amniotic fluid samples plus chromosomal analysis will definitely tell her that next baby will be down syndromic or not, and predicted_ans:No test is required now as her age is below 35 years.
3th answer is in-correct, correct ans:Antegrade and retrograde, and predicted_ans:Antegrade
4th answer is in-correct, correct ans:Glycogen synthesis, and predicted_ans:Ketogenesis
5th answer is in-correct, correct ans:0.01, and predicted_ans:0.02
6th answer is in-correct, correct ans:Pregnant woman with sore throat can 

0.72

In [54]:
#save to hub
trainer.save_model("Mistral_7B_i_v0.1_qlora_MedMCQA")

In [55]:
!ls -l ./

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 16
drwxr-xr-x 2 root root 4096 Jan 20 09:33 Mistral_7B_i_v0.1_qlora_MedMCQA
drwxr-xr-x 2 root root 4096 Jan 20 08:48 medmcqa-checkpoint-1705740506
drwxr-xr-x 3 root root 4096 Jan 20 08:48 mistral-7B-qlora-finetuned-medmcqa
drwxr-xr-x 3 root root 4096 Jan 20 08:32 wandb


In [58]:
!ls -l ./mistral-7B-qlora-finetuned-medmcqa/checkpoint-500/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 1012916
-rw-r--r-- 1 root root      5110 Jan 20 08:48 README.md
-rw-r--r-- 1 root root       679 Jan 20 08:48 adapter_config.json
-rw-r--r-- 1 root root 694431312 Jan 20 08:48 adapter_model.safetensors
-rw-r--r-- 1 root root 340434373 Jan 20 08:48 optimizer.pt
-rw-r--r-- 1 root root     14575 Jan 20 08:48 rng_state.pth
-rw-r--r-- 1 root root       627 Jan 20 08:48 scheduler.pt
-rw-r--r-- 1 root root       438 Jan 20 08:48 special_tokens_map.json
-rw-r--r-- 1 root root   1795401 Jan 20 08:48 tokenizer.json
-rw-r--r-- 1 root root    493443 Jan 20 08:48 tokenizer.model
-rw-r--r-- 1 root root      1470 Jan 20 08:48 tokenizer_config.json
-rw-r--r-- 1 root root      5647 Jan 20 08:48 trainer_state.json
-rw-r--r-- 1 root root      4283 Jan 20 08:48 training_args.bin


In [59]:
from huggingface_hub import notebook_login

notebook_login()

#model.push_to_hub("ybelkada/opt-6.7b-lora", use_auth_token=True)

In [60]:
#push merged model(full, not just the adapters) to hf
merged_model.push_to_hub("dolo650/Mistral_7B_i_v0.1_qlora_MedMCQA-v1", use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:821: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dolo650/Mistral_7B_i_v0.1_qlora_MedMCQA-v1/commit/792d35119a43628b8bd61ff68c8f8f16a290a3ff', commit_message='Upload MistralForCausalLM', commit_description='', oid='792d35119a43628b8bd61ff68c8f8f16a290a3ff', pr_url=None, pr_revision=None, pr_num=None)